In [5]:
import os
import sys
import zarr
import json
import ndjson
from glob import glob

from load_module import read_zarr, get_pos_info

In [6]:
# 指定パス内のディレクトリ名を取得
def get_exp_names(path):
    dir_names = []
    for dir_name in glob(path + "/*"):
        if os.path.isdir(dir_name):
            dir_names.append(dir_name)

    exp_names = []
    for dir_name in dir_names:
        exp_names.append(dir_name.split("/")[-1])

    return exp_names

In [7]:
particle_names = [
    "apo-ferritin",
    "beta-amylase",
    "beta-galactosidase",
    "ribosome",
    "thyroglobulin",
    "virus-like-particle",
]

particle2extrafilename = {
    "apo-ferritin": "ferritin_complex-1.0_orientedpoint.ndjson",
    "beta-amylase": "beta_amylase-1.0_orientedpoint.ndjson",
    "beta-galactosidase": "beta_galactosidase-1.0_orientedpoint.ndjson",
    "ribosome": "cytosolic_ribosome-1.0_orientedpoint.ndjson",
    "thyroglobulin": "thyroglobulin-1.0_orientedpoint.ndjson",
    "virus-like-particle": "pp7_vlp-1.0_orientedpoint.ndjson",
}

particle2dirNumber = {
    "apo-ferritin": "101",
    "beta-amylase": "102",
    "beta-galactosidase": "103",
    "ribosome": "104",
    "thyroglobulin": "105",
    "virus-like-particle": "106",
}

particle2savefilename = {
    "apo-ferritin": "apo-ferritin.json",
    "beta-amylase": "beta-amylase.json",
    "beta-galactosidase": "beta-galactosidase.json",
    "ribosome": "ribosome.json",
    "thyroglobulin": "thyroglobulin.json",
    "virus-like-particle": "virus-like-particle.json",
}

***粒子座標情報の保存***

In [9]:
exp_names = get_exp_names("../inputs/10441/")

for exp_name in exp_names:
    save_base_dir = f"../inputs/10441/{exp_name}/Picks/"

    if not os.path.exists(save_base_dir):
        os.makedirs(save_base_dir)

    for particle_name in particle_names:
        pos_info_path = f"../inputs/10441/{exp_name}/Reconstructions/VoxelSpacing10.000/Annotations/{particle2dirNumber[particle_name]}/{particle2extrafilename[particle_name]}"
        pos_info = get_pos_info(pos_info_path)

        # dict型のpos_infoをjson形式で保存
        save_path = os.path.join(save_base_dir, particle2savefilename[particle_name])
        with open(save_path, "w") as f:
            json.dump(pos_info, f)

***エクストラデータセットの移管***

In [10]:
import shutil

for exp_name in exp_names:
    dir_number = 100
    tomograms_path = f"../inputs/10441/{exp_name}/Reconstructions/VoxelSpacing10.000/Tomograms/{dir_number}/{exp_name}.zarr"
    tomogram_save_dir = (
        f"../inputs/train/static/ExperimentRuns/{exp_name}/VoxelSpacing10.000/"
    )
    tomograms_save_path = os.path.join(tomogram_save_dir, "denoised.zarr")

    if not os.path.exists(tomogram_save_dir):
        os.makedirs(tomogram_save_dir)
        shutil.copytree(tomograms_path, tomograms_save_path)

    # もし存在していたら削除してからコピー
    else:
        shutil.rmtree(tomograms_save_path)
        shutil.copytree(tomograms_path, tomograms_save_path)

In [11]:
for exp_name in exp_names:
    particle_dir = f"../inputs/10441/{exp_name}/Picks/"
    particle_save_dir = f"../inputs/train/overlay/ExperimentRuns/{exp_name}/Picks/"

    if not os.path.exists(particle_save_dir):
        os.makedirs(particle_save_dir)

    # particle_dir内の全てのjsonファイルをparticle_save_dirにコピー
    for particle_name in particle_names:
        particle_path = os.path.join(particle_dir, particle2savefilename[particle_name])
        particle_save_path = os.path.join(
            particle_save_dir, particle2savefilename[particle_name]
        )
        shutil.copy(particle_path, particle_save_path)

In [12]:
def get_exp_names(path):
    dir_names = []
    for dir_name in glob(path + "/*"):
        if os.path.isdir(dir_name):
            dir_names.append(dir_name)

    exp_names = []
    for dir_name in dir_names:
        exp_names.append(dir_name.split("/")[-1])

    return exp_names


exp_names = get_exp_names("../inputs/train/static/ExperimentRuns/")
exp_names.remove("TS_4")